In [8]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd

In [158]:
# import data from pickle file
df = pd.read_pickle('data/users.pkl')
df.head()

,business_address,first_name,gender,haves,languages,last_name,primary_company,wants,company_country,company_street,company_city,company_zipcode,full_name,job_title,company,haves_vector,wants_vector,languages_vector
0,"{'zipcode': '22763', 'country': 'DE', 'city': ...",Clemens,1,[],[],Nill,"{'title': 'Data Engineer', 'name': 'Adler Smar...","[APO SNP, ASW, Administration réseau, Agent De...",DE,Griegstraße 75 /22a,Hamburg,22763,Clemens Nill,Data Engineer,Adler Smart Solutions,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0]"
1,"{'zipcode': '22113', 'country': 'DE', 'city': ...",Patrick,0,[AMRT],[],Wittwer,"{'title': 'Accountant', 'name': 'Steuber Recyc...","[Bodywork, Advanced Materials, Business Recove...",DE,Liebigstraße 82 - 84,Hamburg,22113,Patrick Wittwer,Accountant,Steuber Recycling GmbH,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0]"
2,"{'zipcode': '14055', 'country': 'DE', 'city': ...",Hans,1,"[Business Process Integration, Agency Selectio...","[pt, en]",Kurry,"{'title': 'Professor', 'name': 'Tennisverein S...","[Business Requirements, Agile, Account Directo...",DE,Harbigstraße 40,Berlin,14055,Hans Kurry,Professor,Tennisverein SC Brandenburg e.V.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0]"
3,"{'zipcode': '10777', 'country': 'DE', 'city': ...",Barbara,0,"[A/R Management, Allergy, Abstract Paintings, ...",[en],Nill,"{'title': 'Professor', 'name': 'Mopolis Media ...",[Agriculture],DE,Fuggerstr. 35,Berlin,10777,Barbara Nill,Professor,Mopolis Media GmbH,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0]"
4,"{'zipcode': '14195', 'country': 'DE', 'city': ...",Frank,0,"[ASPX, APICS, Ceramic Analysis, Department Org...",[pt],Schäfer,"{'title': 'Data Scientist', 'name': 'QuadCover...","[AVL, Central Nervous System, Field Technician...",DE,Arnimallee 14,Berlin,14195,Frank Schäfer,Data Scientist,QuadCover Geometry Solutions,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0]"


In [184]:
# create np ndarray from the df
def full_encoding(row):
    return [row['gender']] + row['haves_vector'] + row['wants_vector'] + row['languages_vector']

matrix = df.apply(full_encoding, axis=1).as_matrix()
data = np.zeros(shape=(len(matrix), len(matrix[0])))

# fill array with data taken from the df
for i, row in enumerate(matrix):
    data[i] = np.asarray(row)

In [196]:
# select which column we want to predict
def split_dataset_languages(data, df):
    length = len(df['languages_vector'][0])
    pos = len(data[0]) - length
    return data[:, :pos], data[:, pos:]


x, y = split_dataset_languages(data, df) 
input_dim = x.shape[1]
x.shape, y.shape

((5000, 2005), (5000, 5))

In [200]:
# construct keras model
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=input_dim))
model.add(Dense(units=5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [202]:
model.fit(x, y, validation_split=0.2, epochs=10, batch_size=32)

Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 0s 113us/step - loss: 1.4907 - acc: 0.3940 - val_loss: 1.6418 - val_acc: 0.1620
Epoch 2/10
4000/4000 [==============================] - 0s 110us/step - loss: 1.3307 - acc: 0.4338 - val_loss: 1.6962 - val_acc: 0.1750
Epoch 3/10
4000/4000 [==============================] - 0s 109us/step - loss: 1.1408 - acc: 0.4898 - val_loss: 1.7894 - val_acc: 0.1890
Epoch 4/10
4000/4000 [==============================] - 0s 125us/step - loss: 0.9782 - acc: 0.5145 - val_loss: 1.9099 - val_acc: 0.1940
Epoch 5/10
4000/4000 [==============================] - 0s 107us/step - loss: 0.8525 - acc: 0.5365 - val_loss: 2.0489 - val_acc: 0.2090
Epoch 6/10
4000/4000 [==============================] - 1s 130us/step - loss: 0.7626 - acc: 0.5410 - val_loss: 2.1873 - val_acc: 0.2000
Epoch 7/10
4000/4000 [==============================] - 0s 108us/step - loss: 0.6990 - acc: 0.5558 - val_loss: 2.3402 - val_acc: 0.2040
